In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")



In [3]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model = "gemini-1.0-pro-001")
# llm = ChatGoogleGenerativeAI(model = "gemini-1.0-pro-002")

In [9]:
from langchain_groq import ChatGroq
llm = ChatGroq(model = "Gemma2-9b-IT")

## Simple AI Assistant

In [10]:
while True:
    question = input("Type yourquestion: ")
    if question != "q":
        print(llm.invoke(question).content)
    else:
        print("Goodbye")
        break

Please provide me with a prompt or question so I can assist you. 😊  

For example, you could ask me to:

* **Write a story:** "Write a short story about a robot who learns to feel emotions."
* **Answer a question:** "What is the capital of France?"
* **Summarize text:** "Summarize the plot of the novel *Pride and Prejudice*."
* **Translate language:** "Translate 'Hello, how are you?' into Spanish."
* **Generate creative content:** "Give me some ideas for a birthday party."


I'm ready for anything! ✨ 

Goodbye


## Memory

In [11]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [12]:
store = {}

In [13]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [14]:
config = {
    "configurable": {"session_id": "firstchat"},
}

In [15]:
model_with_memory = RunnableWithMessageHistory(llm , get_session_history)

In [16]:
model_with_memory.invoke(("HI my name is sumay but someone call me with raja"),config = config).content

"Hi Raja! \n\nIt's nice to meet you. 😊 \n\nDo you have any questions for me, or would you like to chat about something in particular? \n\n"

In [17]:
model_with_memory.invoke(("what is my  name"),config = config).content

'Your name is Sumay, but people also call you Raja!  \n\nIs there anything else I can help you with?\n'

In [19]:
while True:
    question = input("Type your question: ")
    if question != "q":
        print(model_with_memory.invoke(question,config = config).content)
    else:
        print("Goodbye")
        break

Your name is Sumay, but people also call you Raja! 😊  

Do you want to talk about something else?

The capital of India is **New Delhi**. 

Is there anything else you'd like to know? 😊  

Goodbye


In [33]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content='HI my name is sumay but someone call me with raja', additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Raja! \n\nIt's nice to meet you. 😊 \n\nDo you have any questions for me, or would you like to chat about something in particular? \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 21, 'total_tokens': 61, 'completion_time': 0.072727273, 'prompt_time': 0.002116796, 'queue_time': 0.015883133, 'total_time': 0.074844069}, 'model_name': 'Gemma2-9b-IT', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-534ad46d-21e6-45e5-9899-eb661d2aecd3-0', usage_metadata={'input_tokens': 21, 'output_tokens': 40, 'total_tokens': 61}), HumanMessage(content='what is my  name', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Sumay, but people also call you Raja!  \n\nIs there anything else I can he

## Tools and Agents

In [35]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [38]:
api_wrapper = WikipediaAPIWrapper()

In [39]:
tool = WikipediaQueryRun(api_wrapper = api_wrapper)   

In [40]:
tool.name

'wikipedia'

In [41]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [42]:
tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [44]:
print(tool.run({"query" : "langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

Page: Retrieval-augmented generation
Summary: Retrieval-augmented generation (RAG) is a technique that grants generative artificial intelligence models information retrieval capabilities. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information to augment information drawn from its own vast, static training data. This allows LLMs to use domain-specific and/or updated information.  
Use cases include providing chatbot access to internal company data or giving factual information only from an authoritative source.

Page: Milvus (vec

In [45]:
print(tool.run({"query" : "About the India"}))

Page: India
Summary: India, officially the Republic of India, is a country in South Asia.  It is the seventh-largest country by area; the most populous country from June 2023 onwards; and since its independence in 1947, the world's most populous democracy. Bounded by the Indian Ocean on the south, the Arabian Sea on the southwest, and the Bay of Bengal on the southeast, it shares land borders with Pakistan to the west; China, Nepal, and Bhutan to the north; and Bangladesh and Myanmar to the east. In the Indian Ocean, India is near Sri Lanka and the Maldives; its Andaman and Nicobar Islands share a maritime border with Thailand, Myanmar, and Indonesia.
Modern humans arrived on the Indian subcontinent from Africa no later than 55,000 years ago. Their long occupation, predominantly in isolation as hunter-gatherers, has made the region highly diverse, second only to Africa in human genetic diversity. Settled life emerged on the subcontinent in the western margins of the Indus river basin 9

In [47]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [48]:
tool3 = TavilySearchResults()

In [49]:
tool3.run({"query" : "History of Jaggarnath Maahapraabhu"})

[{'url': 'https://en.lordjagannath.in/sri-jagannath/the-antiquity-of-jagannath-mahaprabhu',
  'content': "Finally, the legend of the tribal origin of Mahaprabhu. It's believed that Jagannath was worshiped by Sabaras (a caste of hunters) in far-off Koraput district."},
 {'url': 'https://justkalinga.com/in/the-origin-story-of-shri-jagannath-mahaprabhu-part-1/?srsltid=AfmBOooR751hsi3Qf97AIWt6sJGpP3745DWO9tr5ny6dRfQIT6UrXtbF',
  'content': "Embark on a captivating exploration of the origins of Lord Jagannath with this insightful e-book, ``The Origin Story of Shri Jagannath Mahaprabhu Part 1.'' Delve into the rich tapestry of traditions and legends that surround this beloved deity of Odisha."},
 {'url': 'https://yourspositively.com/the-mystery-joy-and-glory-of-mahaprabhu-jagannath/',
  'content': "MANAS DAS' effusive description of the glorious history and mystery of Mahaprabhu Jagannath will evoke joy and divinity within each one of us."},
 {'url': 'https://svastika.in/blogs/blog/who-is-lo

In [50]:
tool3.run({"query" : "What is hari das thakur"})

[{'url': 'https://gaudiyahistory.iskcondesiretree.com/tag/haridasa-thakura/',
  'content': 'Haridas Thakur is the Acharya of the chanting of the Holy Names. He has spread the Holy Name of Krishna in all directions. But now the Lord has taken him away'},
 {'url': 'https://thegaudiyatreasuresofbengal.com/2021/05/01/namacharya-haridas-thakur/',
  'content': 'Srila Haridas Thakur is the namacharya of the Brahma-Madhva-Gaudiya sampradaya and is an eternal associate of Sri Chiatanya Mahaprabhu.'},
 {'url': 'https://www.quora.com/Whats-the-story-of-Haridas-Thakur-in-ancient-history',
  'content': 'Haridas Thakur, also known as Lord Brahma, Prahlada Maharaja, and Mahatapa, is an important figure in the Brahma-Madhva-Gaudiya sampradaya.'},
 {'url': 'https://en.wikipedia.org/wiki/Haridasa_Thakur',
  'content': 'Haridasa Thakura was a devotee of the deity Krishna, and is regarded to have practised the chant of his veneration, the Hare Krishna mantra, 300,000 times daily'},
 {'url': 'https://www.r

In [51]:
tool.run({"query" : "Name all the sampradayas in hinduism"})

'Page: Sampradaya\nSummary: Sampradaya (/səmpɾəd̪ɑjə/,Sanskrit: सम्प्रदाय; IAST: Saṃpradāya), in Indian-origin religions, namely Hinduism, Buddhism, Jainism, and Sikhism, can be translated as \'tradition\', \'spiritual lineage\', \'sect\', or \'religious system\'. To ensure continuity and transmission of dharma, various sampradayas have the Guru-shishya parampara in which parampara or lineage of successive gurus (masters) and shishyas (disciples) serves as a spiritual channel and provides a reliable network of relationships that lends stability to a religious identity. Shramana is vedic term for seeker or shishya. Identification with and followership of sampradayas is not static, as sampradayas allows flexibility where one can leave one sampradaya and enter another or practice religious syncretism by simultaneously following more than one sampradaya. Samparda is a Punjabi language term, used in Sikhism, for sampradayas.\n\nPage: Daśanāmi Sampradaya\nSummary: The Daśanāmi Sampradaya (IA